In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd

In [4]:
# load dataset

data = pd.read_csv('dataset/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# here we need to calculate the salary , estimated salary is my target
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [8]:
# convert gender category into numerical values
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
# onehot encoder
label_encoder_geo = OneHotEncoder()
geo_encoded = label_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
# data = pd.concat([data.reset_index(drop=True), geo_encoded_df], axis=1)
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# saving the label encoder files as pickle file
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('label_encoder_geo.pkl', 'wb') as file:
    pickle.dump(label_encoder_geo, file)

In [11]:
# split x and y 
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [12]:
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0


In [13]:
y.head()

0    101348.88
1    112542.58
2    113931.57
3     93826.63
4     79084.10
Name: EstimatedSalary, dtype: float64

In [14]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
scaler.feature_names_in_

array(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [16]:
with open('scaler_salary_reg.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [40]:
# ann regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras

In [52]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # if we didnt add the activation function, it would be linear
])

# model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [54]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='mean_absolute_error',metrics=['mae'])

In [55]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                832       
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
# set up tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [57]:
# setup early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

In [58]:
# train the model
history = model.fit(
    X_train, y_train, 
    epochs=100, 
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback, early_stopping_callback],
    )

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 100367.2734 - mae: 100367.2734 - val_loss: 98480.3516 - val_mae: 98480.3516
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99500.7188 - mae: 99500.7188 - val_loss: 96742.1953 - val_mae: 96742.1953
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 96484.9141 - mae: 96484.9141 - val_loss: 92365.5859 - val_mae: 92365.5859
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 90638.8984 - mae: 90638.8984 - val_loss: 85160.7031 - val_mae: 85160.7031
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 82250.6562 - mae: 82250.6562 - val_loss: 76134.5156 - val_mae: 76134.5156
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 72739.3047 - mae: 72739.3047 - val_loss: 67138.0859 - val_mae: 67138.0859
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 63966.5391 - mae: 63966.5391 

In [59]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [60]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 10015), started 13:27:53 ago. (Use '!kill 10015' to kill it.)

In [61]:
# evaluate the model 
test_loss , test_mae = model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')customer

63/63 [==============================] - 0s 3ms/step - loss: 50349.5078 - mae: 50349.5078
Test MAE : 50349.5078125


In [62]:
# save the model
model.save('regression_model.h5')

/home/akshay/anaconda3/envs/churn_ann/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [63]:
input_data = pd.DataFrame({
    'CreditScore': [600],
    'Gender': [0],
    'Age': [40],
    'Tenure': [3],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'Exited': [0],
    'Geography_France': [1],
    'Geography_Germany': [0],
    'Geography_Spain': [0]
})
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,0,1,0,0


In [64]:
scaled_input_data = scaler.transform(input_data)
scaled_input_data

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.50857963,  1.00150113,
        -0.57946723, -0.57638802]])

In [67]:
prediction = model.predict(scaled_input_data)
prediction

1/1 [==============================] - 0s 27ms/step


array([[90987.53]], dtype=float32)

In [68]:
prediction_proba = prediction[0][0] 
print('customer salary is : ', prediction_proba)

customer salary is :  90987.53
